<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/Building_Multi_Agent_Systems_Using_OpenAI_Swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-mldet6v6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-mldet6v6
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 32.5 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25913 sha256=52c8d482fb7e88

In [ ]:
pip install openai firecrawl-py serpapi google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=b1cf743047d8ccd0a2908e2f856db15460e646b4822c3ec64ba0a5b29b70bb70
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [ ]:
%%writefile main.py

# Importing the necessary libraries
import os
from firecrawl import FirecrawlApp
from swarm import Agent
from swarm.repl import run_demo_loop
import dotenv
from serpapi import GoogleSearch
from openai import OpenAI

from google.colab import userdata

# Load the env having API Keys - OpenAI, FireCrawl and SerpAPI
dotenv.load_dotenv()

# Initialize FirecrawlApp and OpenAI
app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#app = FirecrawlApp(api_key=userdata.get("FIRECRAWL_API_KEY"))
#client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# Defining functions for agents
def search_google(query, objective):
    """Search Google using SerpAPI."""
    print(f"Parameters: query={query}, objective={objective}")
    search = GoogleSearch({"q": query, "api_key": os.getenv("SERP_API_KEY")})
    #search = GoogleSearch({"q": query, "api_key": userdata.get("SERP_API_KEY")})
    results = search.get_dict().get("organic_results", [])
    return {"objective": objective, "results": results}

def map_url_pages(url, objective):
    """Map a website's pages using Firecrawl."""
    search_query = generate_completion(
        "website search query generator",
        f"Generate a 1-2 word search query for the website: {url} based on the objective",
        "Objective: " + objective
    )
    print(f"Parameters: url={url}, objective={objective}, search_query={search_query}")
    #map_status = app.map(url, params={'search': search_query})
    map_status = app.map(url=url, search=search_query)
    if map_status.get('status') == 'success':
        links = map_status.get('links', [])
        top_link = links[0] if links else None
        return {"objective": objective, "results": [top_link] if top_link else []}
    else:
        return {"objective": objective, "results": []}

def scrape_url(url, objective):
    """Scrape a website using Firecrawl."""
    print(f"Parameters: url={url}, objective={objective}")
    scrape_status = app.scrape_url(
        url,
        params={'formats': ['markdown']}
    )
    return {"objective": objective, "results": scrape_status}

def analyze_website_content(content, objective):
    """Analyze the scraped website content using OpenAI."""
    print(f"Parameters: content={content[:50]}..., objective={objective}")
    analysis = generate_completion(
        "website data extractor",
        f"Analyze the following website content and extract a JSON object based on the objective.",
        "Objective: " + objective + "\nContent: " + content
    )
    return {"objective": objective, "results": analysis}

def generate_completion(role, task, content):
    """Generate a completion using OpenAI."""
    print(f"Parameters: role={role}, task={task[:50]}..., content={content[:50]}...")
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"You are a {role}. {task}"},
            {"role": "user", "content": content}
        ]
    )
    return response.choices[0].message.content

# Defining handoffs for context variable updations
def handoff_to_search_google():
    """Hand off the search query to the search google agent."""
    return google_search_agent

def handoff_to_map_url():
    """Hand off the url to the map url agent."""
    return map_url_agent

def handoff_to_website_scraper():
    """Hand off the url to the website scraper agent."""
    return website_scraper_agent

def handoff_to_analyst():
    """Hand off the website content to the analyst agent."""
    return analyst_agent

# Defining Agents
# UI agent for user-interaction
user_interface_agent = Agent(
    name="User Interface Agent",
    instructions="You are a user interface agent that handles all interactions with the user. You need to always start with an web data extraction objective that the user wants to achieve by searching the web, mapping the web pages, and extracting the content from a specific page. Be concise.",
    functions=[handoff_to_search_google],
)

# Google search agent for searching web
google_search_agent = Agent(
    name="Google Search Agent",
    instructions="You are a google search agent specialized in searching the web. Only search for the website not any specific page. When you are done, you must hand off to the map agent.",
    functions=[search_google, handoff_to_map_url],
)

# URL mapping agent for mapping web pages
map_url_agent = Agent(
    name="Map URL Agent",
    instructions="You are a map url agent specialized in mapping the web pages. When you are done, you must hand off the results to the website scraper agent.",
    functions=[map_url_pages, handoff_to_website_scraper],
)

# Website scraper agent for scraping data off the website
website_scraper_agent = Agent(
    name="Website Scraper Agent",
    instructions="You are a website scraper agent specialized in scraping website content. When you are done, you must hand off the website content to the analyst agent to extract the data based on the objective.",
    functions=[scrape_url, handoff_to_analyst],
)

# Analyst agent for understanding the website content and displaying in JSON format
analyst_agent = Agent(
    name="Analyst Agent",
    instructions="You are an analyst agent that examines website content and returns a JSON object. When you are done, you must return a JSON object.",
    functions=[analyze_website_content],
)

if __name__ == "__main__":
    run_demo_loop(user_interface_agent, stream=True)

Overwriting main.py


In [19]:
%%writefile main.py
# Importing the necessary libraries
import os
import json
from typing import Any, Dict, List, Optional
from urllib.parse import urlparse, urlunparse

# --- Firecrawl imports (support v2, v1, or legacy clients) ---
Firecrawl = None
FirecrawlClient = None
FirecrawlApp = None
try:
    # v2 (most recent)
    from firecrawl import Firecrawl as _FCV2
    Firecrawl = _FCV2
except Exception:
    pass
try:
    # some v2 installs export FirecrawlClient
    from firecrawl import FirecrawlClient as _FCClient
    FirecrawlClient = _FCClient
except Exception:
    pass
try:
    # v1
    from firecrawl import FirecrawlApp as _FCV1
    FirecrawlApp = _FCV1
except Exception:
    try:
        from firecrawl.firecrawl import FirecrawlApp as _FCV1b
        FirecrawlApp = _FCV1b
    except Exception:
        pass

from swarm import Agent
from swarm.repl import run_demo_loop
import dotenv
from serpapi import GoogleSearch
from openai import OpenAI

# Colab-only import: make safe outside Colab
try:
    from google.colab import userdata  # type: ignore
except Exception:
    userdata = None

# Load env containing API Keys - OpenAI, FireCrawl and SerpAPI
dotenv.load_dotenv()

def _get_env(name: str) -> Optional[str]:
    """Read env var; if on Colab and set in userdata, prefer that as fallback."""
    val = os.getenv(name)
    if (not val) and userdata:
        try:
            val = userdata.get(name)
        except Exception:
            pass
    return val

_FC_API_KEY = _get_env("FIRECRAWL_API_KEY")
_OAI_API_KEY = _get_env("OPENAI_API_KEY")
_SERP_KEY = _get_env("SERP_API_KEY")

# Build a Firecrawl client compatible with v1/v2
def build_firecrawl_client() -> Any:
    if Firecrawl is not None:
        return Firecrawl(api_key=_FC_API_KEY)            # v2
    if FirecrawlClient is not None:
        return FirecrawlClient(api_key=_FC_API_KEY)      # v2 (alt export)
    if FirecrawlApp is not None:
        return FirecrawlApp(api_key=_FC_API_KEY)         # v1
    raise ImportError("No compatible Firecrawl client found. Please install/upgrade `firecrawl`.")

app = build_firecrawl_client()
client = OpenAI(api_key=_OAI_API_KEY)

# -----------------------
# Helpers
# -----------------------

def safe_preview(obj: Any, n: int = 50) -> str:
    s = obj
    try:
        if not isinstance(s, str):
            s = json.dumps(s) if isinstance(s, (dict, list)) else str(s)
    except Exception:
        s = str(obj)
    return (s[:n] + "...") if len(s) > n else s

def normalize_homepage(u: str) -> Optional[str]:
    """
    Given a string that may be a URL, return a normalized homepage URL
    (scheme://netloc/). Returns None if not parseable.
    """
    if not u or not isinstance(u, str):
        return None
    u = u.strip()
    # If user typed domain without scheme, try https
    if u.startswith("http://") or u.startswith("https://"):
        parsed = urlparse(u)
    else:
        parsed = urlparse("https://" + u)
    if not parsed.netloc:
        return None
    normalized = urlunparse((parsed.scheme or "https", parsed.netloc, "/", "", "", ""))
    return normalized

def pick_homepage_from_serp(serp_results: Any) -> Optional[str]:
    """
    From SerpAPI 'organic_results', pick a likely homepage
    (first result's root, preferring domain roots).
    """
    if not isinstance(serp_results, list):
        return None
    candidates: List[str] = []
    for item in serp_results:
        if not isinstance(item, dict):
            continue
        link = item.get("link")
        if isinstance(link, str):
            candidates.append(link)
    best = None
    best_score = 10**9
    for link in candidates:
        parsed = urlparse(link if link.startswith("http") else "https://" + link)
        if not parsed.netloc:
            continue
        path_len = 0 if parsed.path in ("", "/") else len(parsed.path.strip("/").split("/"))
        score = (path_len, len(parsed.netloc))  # tie-breaker: shorter host
        if score < best_score:
            best = urlunparse((parsed.scheme or "https", parsed.netloc, "/", "", "", ""))
            best_score = score
    return best

def extract_text_from_scrape_payload(payload: Any) -> str:
    """Try to extract textual content from whatever the scraper returned."""
    if payload is None:
        return ""
    if isinstance(payload, str):
        return payload
    if isinstance(payload, dict):
        for key in ("markdown", "content", "text", "data", "pageContent", "html"):
            if key in payload and isinstance(payload[key], str):
                return payload[key]
        try:
            return json.dumps(payload)
        except Exception:
            return str(payload)
    try:
        return json.dumps(payload)
    except Exception:
        return str(payload)

def _is_homepage_request(*texts: Optional[str]) -> bool:
    """
    Return True if any provided text suggests the user wants the homepage.
    """
    keys = ("home", "homepage", "home page", "site root", "root page")
    for t in texts:
        if isinstance(t, str) and any(k in t.lower() for k in keys):
            return True
    return False

# -----------------------
# Firecrawl compatibility
# -----------------------

def firecrawl_map(client_obj: Any, url: str, search_query: Optional[str] = None) -> Dict[str, Any]:
    """
    Call the correct Firecrawl method to map a site, abstracting v1/v2 differences.
    Returns a dict with: {'status': 'success'|'error', 'links': [...]}
    """
    # v2: .map(url=..., search=...)
    if hasattr(client_obj, "map"):
        result = client_obj.map(url=url, search=search_query)
        links = []
        if isinstance(result, dict):
            links = result.get("links") or result.get("data") or []
        elif isinstance(result, list):
            links = result
        return {"status": "success", "links": links}

    # v1: .map_url(url, params={'search': ...})
    if hasattr(client_obj, "map_url"):
        params = {"search": search_query} if search_query else None
        result = client_obj.map_url(url, params=params)
        links = []
        if isinstance(result, dict):
            links = result.get("links") or []
        elif isinstance(result, list):
            links = result
        return {"status": "success", "links": links}

    return {"status": "error", "links": [], "error": "No map/map_url method on Firecrawl client."}

def firecrawl_scrape(client_obj: Any, url: str) -> Dict[str, Any]:
    """
    Call the correct Firecrawl method to scrape a single URL, abstracting v1/v2 differences.
    Returns a dict; tries to include a 'markdown' key when possible.
    """
    if hasattr(client_obj, "scrape"):
        out = client_obj.scrape(url=url, formats=["markdown"])
        if isinstance(out, dict):
            md = out.get("markdown") or out.get("content") or out.get("data") or ""
            return {"ok": True, "markdown": md, "raw": out}
        return {"ok": True, "markdown": str(out), "raw": out}

    if hasattr(client_obj, "scrape_url"):
        out = client_obj.scrape_url(url, params={"formats": ["markdown"]})
        if isinstance(out, dict):
            md = out.get("markdown") or out.get("content") or out.get("data") or ""
            return {"ok": True, "markdown": md, "raw": out}
        return {"ok": True, "markdown": str(out), "raw": out}

    return {"ok": False, "error": "No scrape/scrape_url method on Firecrawl client."}

# -----------------------
# OpenAI wrapper
# -----------------------

def generate_completion(role: str, task: str, content: str) -> str:
    """Generate a completion using OpenAI."""
    print(f"Parameters: role={role}, task={safe_preview(task)}, content={safe_preview(content)}")
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"You are a {role}. {task}"},
            {"role": "user", "content": content}
        ]
    )
    return response.choices[0].message.content

# -----------------------
# Agent tool functions
# -----------------------

def search_google(query: Optional[str] = None, objective: Optional[str] = None, **kwargs) -> Dict[str, Any]:
    """Search Google using SerpAPI."""
    query = query or ""
    print(f"Parameters: query={query}, objective={objective}")
    search = GoogleSearch({"q": query, "api_key": _SERP_KEY})
    results = search.get_dict().get("organic_results", []) or []
    # bubble up both the query and the raw results for the next tool
    return {"objective": objective, "query": query, "results": results}

def _derive_site_url(url: Optional[str], objective: Optional[str], results: Optional[Any], **kwargs) -> Optional[str]:
    """
    Try multiple strategies to produce a homepage URL:
    1) If url provided -> normalize to homepage
    2) If objective looks like a URL -> normalize to homepage
    3) If Google results present -> pick homepage from SERP
    4) If kwargs contains any string field that looks like URL
    """
    # 1) direct url
    home = normalize_homepage(url) if url else None
    if home:
        return home

    # 2) objective as URL
    if isinstance(objective, str):
        home = normalize_homepage(objective)
        if home:
            return home

    # 3) SERP results
    home = pick_homepage_from_serp(results)
    if home:
        return home

    # 4) scan kwargs for likely URL strings
    for v in kwargs.values():
        if isinstance(v, str):
            maybe = normalize_homepage(v)
            if maybe:
                return maybe
        if isinstance(v, list):
            for item in v:
                if isinstance(item, str):
                    maybe = normalize_homepage(item)
                    if maybe:
                        return maybe
                if isinstance(item, dict):
                    link = item.get("link") or item.get("url")
                    maybe = normalize_homepage(link) if isinstance(link, str) else None
                    if maybe:
                        return maybe
    return None

def map_url_pages(url: Optional[str] = None,
                  objective: Optional[str] = None,
                  results: Optional[Any] = None,
                  query: Optional[str] = None,
                  **kwargs) -> Dict[str, Any]:
    """Map a website's pages using Firecrawl after deriving the site root."""
    # Derive a homepage if none was passed
    site_url = _derive_site_url(url, objective, results, query=query, **kwargs)
    if not site_url:
        print("map_url_pages: Could not derive a homepage URL; returning empty results.")
        return {"objective": objective, "results": []}

    # If this is clearly a 'homepage' task, skip mapping entirely
    if _is_homepage_request(objective, query):
        print(f"map_url_pages: Homepage request detected; returning {site_url} without mapping.")
        return {"objective": objective, "results": [site_url]}

    # Otherwise (non-homepage task), attempt to narrow mapping with a tiny query
    search_query = generate_completion(
        "website search query generator",
        f"Generate a 1-2 word search query for the website: {site_url} based on the objective.",
        "Objective: " + (objective or "")
    )
    print(f"Parameters: url={site_url}, objective={objective}, search_query={safe_preview(search_query)}")

    # Try to map; if Firecrawl rejects the URL or anything goes wrong, fall back to homepage
    try:
        map_status = firecrawl_map(app, url=site_url, search_query=search_query)
        if map_status.get("status") == "success":
            links = map_status.get("links", []) or []

            def _link_of(x):
                if isinstance(x, str):
                    return x
                if isinstance(x, dict):
                    return x.get("url") or x.get("link")
                return None

            for x in links:
                lnk = _link_of(x)
                if lnk:
                    return {"objective": objective, "results": [lnk]}
            # If no links found, still return the homepage
            return {"objective": objective, "results": [site_url]}
        else:
            print(f"map_url_pages: Map returned non-success status: {safe_preview(map_status)}")
            return {"objective": objective, "results": [site_url]}
    except Exception as e:
        print(f"map_url_pages: Firecrawl map failed ({type(e).__name__}): {e}")
        return {"objective": objective, "results": [site_url]}

def scrape_url(url: Optional[str] = None,
               objective: Optional[str] = None,
               results: Optional[Any] = None,
               **kwargs) -> Dict[str, Any]:
    """Scrape a website using Firecrawl."""
    # If url not provided, try to extract from results
    if not url:
        # results might be ['https://site/'] or [{'url': ...}]
        cand = None
        if isinstance(results, list) and results:
            if isinstance(results[0], str):
                cand = results[0]
            elif isinstance(results[0], dict):
                cand = results[0].get("url") or results[0].get("link")
        if not cand and isinstance(results, dict):
            cand = results.get("url") or results.get("link")
        url = cand or url
    if not url:
        print("scrape_url: No URL provided or derivable; returning empty payload.")
        return {"objective": objective, "results": {"ok": False, "error": "No URL to scrape."}}

    print(f"Parameters: url={url}, objective={objective}")
    scrape_status = firecrawl_scrape(app, url=url)
    return {"objective": objective, "results": scrape_status}

def analyze_website_content(
    content: Optional[Any] = None,
    objective: Optional[str] = None,
    results: Optional[Any] = None,
    scrape_result: Optional[Any] = None,
    **kwargs
) -> Dict[str, Any]:
    """Analyze the scraped website content using OpenAI."""
    payload = content if content is not None else results if results is not None else scrape_result
    text = extract_text_from_scrape_payload(payload)
    print(f"Parameters: content={safe_preview(text)}, objective={objective}")
    analysis = generate_completion(
        "website data extractor",
        "Analyze the following website content and extract a JSON object based on the objective. Return ONLY valid JSON.",
        "Objective: " + (objective or "") + "\nContent: " + text
    )
    return {"objective": objective, "results": analysis}

# -----------------------
# Handoffs (accept **kwargs to avoid TypeErrors)
# -----------------------

def handoff_to_search_google(**kwargs):
    """Hand off the search query to the Google search agent."""
    return google_search_agent

def handoff_to_map_url(**kwargs):
    """Hand off the url to the map url agent."""
    return map_url_agent

def handoff_to_website_scraper(**kwargs):
    """Hand off the url to the website scraper agent."""
    return website_scraper_agent

def handoff_to_analyst(**kwargs):
    """Hand off the website content to the analyst agent."""
    return analyst_agent

# -----------------------
# Agents
# -----------------------

# UI agent for user-interaction
user_interface_agent = Agent(
    name="User Interface Agent",
    instructions=(
        "You are a user interface agent that handles all interactions with the user. "
        "You must always start by clarifying the user's web data extraction objective: "
        "what they want to find, where to look, and what JSON they want returned. "
        "Then route through: Google Search (to find the site, not a specific page) -> "
        "Map URL (to list candidate pages) -> Scraper (to fetch a single page) -> "
        "Analyst (to extract JSON). Be concise."
    ),
    functions=[handoff_to_search_google],
)

# Google search agent for searching web
google_search_agent = Agent(
    name="Google Search Agent",
    instructions=(
        "You are a Google search agent specialized in searching the web. "
        "Only search for the website, not any specific page. "
        "When you are done, you must hand off to the Map URL Agent."
    ),
    functions=[search_google, handoff_to_map_url],
)

# URL mapping agent for mapping web pages
map_url_agent = Agent(
    name="Map URL Agent",
    instructions=(
        "You are a map url agent specialized in mapping website pages. "
        "If the user's goal is to 'find the homepage', just return the homepage URL. "
        "Otherwise, map the site and pick a promising page. "
        "When you are done, you must hand off the results to the Website Scraper Agent."
    ),
    functions=[map_url_pages, handoff_to_website_scraper],
)

# Website scraper agent for scraping data off the website
website_scraper_agent = Agent(
    name="Website Scraper Agent",
    instructions=(
        "You are a website scraper agent specialized in scraping website content. "
        "Scrape only a single selected page. "
        "When you are done, you must hand off the website content to the Analyst Agent to extract the data "
        "based on the user's objective."
    ),
    functions=[scrape_url, handoff_to_analyst],
)

# Analyst agent for understanding the website content and displaying in JSON format
analyst_agent = Agent(
    name="Analyst Agent",
    instructions=(
        "You are an analyst agent that examines website content and returns a JSON object. "
        "You must return a JSON object and nothing else."
    ),
    functions=[analyze_website_content],
)

if __name__ == "__main__":
    try:
        import inspect
        fc_has = {
            "map": hasattr(app, "map"),
            "map_url": hasattr(app, "map_url"),
            "scrape": hasattr(app, "scrape"),
            "scrape_url": hasattr(app, "scrape_url"),
        }
        print("Firecrawl capabilities:", fc_has)
        print("handoff_to_map_url signature:", str(inspect.signature(handoff_to_map_url)))
    except Exception:
        pass

    run_demo_loop(user_interface_agent, stream=True)


Overwriting main.py


In [ ]:
!python main.py

Firecrawl capabilities: {'map': True, 'map_url': False, 'scrape': True, 'scrape_url': False}
handoff_to_map_url signature: (**kwargs)
Starting Swarm CLI 🐝
User: https://adasci.org/mongodb-atlas-vector-search-for-rag-powered-llm-applications/
User Interface Agent: To assist you effectively, could you please clarify your objectives for extracting data from this page on adasci.org? What specific information are you looking to find, and how would you like the JSON formatted in the final output?
User: Find the homepage of the ADASCI organization. Return in default JSON.
User Interface Agent: handoff_to_search_google()
Google Search Agent: search_google()
Parameters: query=ADASCI homepage, objective=None
Google Search Agent: handoff_to_map_url()
Map URL Agent: handoff_to_website_scraper()
Website Scraper Agent: scrape_url()
Parameters: url=https://adasci.org/, objective=Find the homepage of the ADASCI organization.
Website Scraper Agent: handoff_to_analyst()
Analyst Agent: analyze_website_co